в данном ноутбуке мы разбиваем обучающую выборку на много частей чтобы потом было проще загружать

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import datetime
import sklearn
import sqlalchemy as sa
from tqdm import tqdm, tqdm_notebook
import pickle
import matplotlib.pyplot as plt
import itertools
import os

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
%matplotlib inline

In [9]:
# создание коннекта к БД и крурсора для запросов
conn = sqlite3.connect('../data/input/user_info_HSE_hashed.sqlite')
c = conn.cursor()

# создание engine для чтения данных в DataFrame
engine = sa.create_engine('sqlite:///../data/input/user_info_HSE_hashed')

In [10]:
# пример получения перечня таблиц (в данном случае она будет 1)
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print ('Перечень таблиц: {0}\r\n\r\n'.format(c.fetchall()))

# перечень столбцов
c.execute("SELECT * FROM rsa_event_log LIMIT 1;")
cols = [descr[0] for descr in c.description]
print ('Столбцы в таблице: {0}'.format(cols))

Перечень таблиц: [('rsa_event_log',)]


Столбцы в таблице: ['cdf_s_123', 'cdf_s_124', 'cdf_s_127', 'cdf_s_135', 'cdf_s_130', 'cdf_s_129', 'cdf_s_134', 'cdf_s_128', 'cdf_s_138', 'cdf_s_126', 'cdf_s_133', 'cdf_s_136', 'cdf_s_137', 'cdf_s_140', 'cdf_s_178_hashed', 'cdf_s_19', 'cdf_s_20', 'cdf_s_218', 'cdf_s_294', 'cdf_s_299', 'amount_original', 'channel_indicator_desc', 'data_i_118', 'data_i_119', 'data_i_120', 'data_i_154', 'data_s_65', 'event_description', 'event_id', 'event_time', 'ext_acct_number_hashed', 'hardwareid', 'short_date', 'user_acct_number_hashed', 'user_agent_string_hash', 'browser_plugins_hash', 'screen_hash', 'user_id', 'ip_address', 'ip_country', 'ip_region', 'ip_city', 'ip_isp']


In [11]:
# Пример форомирования DataFrame из БД с заполнением типа полей

def transform_cols (df, dict_col_types = None):
    # Расширяйте для необходимых столбцов и их явной типизации
    if dict_col_types is None:
        dict_col_types = {
        'amount_original':(float, 0.0),
        'cdf_s_126':(str, u'null'),
        'cdf_s_138':(str, u'null'),
        'channel_indicator':(str, u'null'),
        'event_description':(str, u'null'),
        'cdf_s_294':(int, 0),
        'cdf_s_140':(float, 0.0),
        'data_i_120':(int, 0),
        'cdf_s_218':(str, u'null'),
        'data_s_65':(int, 0),
        'cdf_s_127':(int, 30),
        'cdf_s_135':(int, 30),
        'cdf_s_130':(int, 30),
        'cdf_s_129':(int, 30),
        'cdf_s_134':(int, 30),
        'data_i_154':(float, np.nan),
        'cdf_s_133':(int, 30),
        'cdf_s_20':(str, u'null'),
        'cdf_s_299':(str, u'null'),
        }
                
    if df.shape[0] > 0:
        df.replace(u'null', np.nan, inplace=True)

        for i in dict_col_types:
            if i in df.columns:
                df[i] = df[i].fillna(dict_col_types[i][1]).astype(dict_col_types[i][0])
    
    return df
    

def df_from_sql (sql, parse_dates = ['event_time','cdf_s_19', 'cdf_s_123','cdf_s_124'], dict_col_types = None, chunk_size = None, engine=engine):
    """
    Функция вощвращающая df в результате переданного sql-запроса
    Также поддерживается приведение столбцов к заданным типам 
    и чтение по блокам
    """
     
    if chunk_size is not None:
        df_iter =  [transform_cols(chunk, dict_col_types) for chunk in pd.read_sql_query(sql, engine, parse_dates = parse_dates, chunksize=chunk_size)]
    else:
        df_iter = transform_cols(pd.read_sql_query(sql, engine, parse_dates = parse_dates), dict_col_types)
        #df_iter = pd.read_sql_query(sql, engine, con=conn,  parse_dates = parse_dates)
    
    return df_iter


-----

In [12]:
all_users = df_from_sql('select DISTINCT user_id from rsa_event_log;',
                        engine=conn,
                        parse_dates=None)

In [13]:
rng = np.random.RandomState(seed = 1)

In [14]:
all_users_list = sorted(all_users['user_id'].astype(str).tolist())

In [15]:
rng.shuffle(all_users_list)

In [16]:
all_users_list[:5]

['MBK14703099', '4196086', '58804548', '32732927', '135593']

In [17]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [18]:
len(all_users_list)

85764

In [20]:
user_ids_chunks = list(chunks(all_users_list, len(all_users_list) // 25))

In [21]:
with open('../data/raw_splits/test/user_ids_chunks.pkl', 'wb') as f:
    pickle.dump(user_ids_chunks, f)

In [22]:
folder = '../data/raw_splits/test/'

In [23]:
for n, chunk in tqdm_notebook(list(enumerate(user_ids_chunks))):
    sql_query_str = "select * from rsa_event_log where user_id in ({0});".format(str(chunk)[1:-1])
    data = pd.read_sql_query(sql_query_str, con=conn, 
                             parse_dates = ['event_time','cdf_s_19', 'cdf_s_123','cdf_s_124'])
    fname = os.path.join(folder, f'chunk_{n}.fth')
    data.to_feather(fname)